# CV Enhancer Prototype

This notebook develops a prototype agent that enhances work experiences in a resume using generative language models.

To use this notebook, make sure you have the necessary dependencies installed and configure the appropriate environment variables to access generative language model services.

You need to install the following libraries if you don't already have them (use uv to install dependencies):

```bash
uv venv
uv pip install -e .
```



Additionally, you need to authenticate with the Gemini API. To do this, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys) and add the environment variable `GOOGLE_API_KEY` to a `.env` file in the root of your project. Ensure that the `.env` file contains the following line:

```
GOOGLE_API_KEY=tu_api_key_aqui
```

## 1. ⚙️ Initial Configuration and Setup
This section covers the environment preparation, constant definitions, and the critical step of initializing the core components of the Agent system: the Large Language Models (LLMs) and their corresponding configurations.

### 🛠️ Environment Initialization and Global Constraints

This block initializes the project environment and defines global constants that govern the agent's behavior and operational scope.

* **Environment Loading**: Securely loads environment variables (e.g., API keys) using `dotenv`. This ensures separation of credentials from the codebase.

In [1]:
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

True

### Design Constraints & Scope Configuration

Defines the operational boundaries of the prototype to manage computational costs and latency. 

* **`EXPERIENCE_LIMITS` (Architectural Design)**: Defines a dictionary to specify the **maximum number of experiences to enhance per category** (e.g., `work=1`). This limits the scope of the RAG and enhancement process, prioritizing the most recent or relevant experiences and optimizing API usage.
* **`MAX_REFINEMENT_ITERATIONS`**: Sets the cap for the number of internal loops the **Refinement Agent** will run when seeking improvement feedback from the **Critic Agent**. This directly controls the trade-off between output quality and processing cost/latency.

In [2]:
import typing as t


class ExperienceLimits(t.TypedDict, total=False):
    """Limits for the number of experiences to enhance in each category."""

    work: int
    volunteer: int
    certificates: int
    projects: int
    skills: int
    interests: int


EXPERIENCE_LIMITS = ExperienceLimits(
    work=3,
    projects=1,
)

MAX_REFINEMENT_ITERATIONS = 1

### 📂 Input Data Definition

This cell defines the file path for the candidate's CV and imports the target job description text, which serves as the primary input for the entire enhancement pipeline.

* **`JSON_RESUME_FILE`**: Path to the candidate's original CV, formatted in the `JsonResume` standard.
* **`JOB_OFFER_TEXT`**: The raw text of the job advertisement. This text will be processed by the first agent in the chain (`JobOfferAnalyzerAgent`) to extract actionable requirements.

In [3]:
JSON_RESUME_FILE = Path("../data/json_resume_example.json")

# Offer obtained from: https://www.linkedin.com/jobs/collections/?currentJobId=4324487265
JOB_OFFER_TEXT = """
We are still looking for talent… and we would love for you to join our team!

For over 25 years, UST has worked alongside the world's best companies to make a real impact through business transformation. Driven by technology, inspired by people, and guided by our purpose, UST supports clients from design to implementation. Together, with more than 30,000 employees in 30 countries, we build to create limitless impact, reaching billions of lives in the process.


We are looking for an AI Engineer, to join a strategic project supporting data platform modernization.



UST is looking for a candidate with strong Python expertise, proven experience building applications using LLMs, and hands-on exposure to agentic AI frameworks.



What We're Looking For:

Experience: 6 to 8 years of professional experience.
Language: Advanced English B2 - C1
Strong programming expertise in Python.
Proven experience building applications using any LLMs.
1 to 2 years of hands-on experience with agentic AI frameworks.
Familiarity with cloud platforms.


Why Join Us:

Work in a remote and global environment
Be part of a mission-critical application team
Work in a supportive, collaborative environment
Opportunity to mentor others and influence product direction
Gain exposure to cloud migration and modern development practices


UST is waiting for you!
"""

### 🧠 Model Configuration and LLM Strategy

This section defines the retry policy and initializes the specific **Gemini LLM models** used by each component of the multi-agent system. This demonstrates a **strategic model selection** based on the task complexity.

* **`retry_config`**: Defines robust HTTP retry options (e.g., retrying on status codes 429, 503) to ensure resilience against temporary API throttling or server errors.
* **`CvSaverConfig` (Design Pattern)**: A dataclass used to encapsulate and manage the different LLM instances required by the various agents, promoting **clean architecture** and easy configuration swap.
* **Model Selection Strategy**:
    * **`gemini-2.5-pro` (Critic Agent)**: The Pro model is selected for the `experience_critique_model` due to its superior reasoning and analytical capabilities, which are essential for providing high-quality, constructive feedback during the refinement loop.
    * **`gemini-2.5-flash-preview-09-2025` (Summarizer & Enhancer)**: The Flash model is used for data extraction and initial content generation tasks where speed and cost efficiency are prioritized.
    * **`gemini-2.5-flash-lite` (Query Builder)**: The lightest model is used for the `experience_query_builder_model` as the task involves concise output generation, maximizing cost savings.

In [4]:
from dataclasses import dataclass
from pathlib import Path

from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=100,  # Maximum retry attempts
    initial_delay=1,
    max_delay=60,  # Maximum delay between retries
    exp_base=3,  # Delay multiplier
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


@dataclass
class CvSaverConfig:
    job_offer_analyzer_model: Gemini
    experience_refiner_model: Gemini
    experience_critique_model: Gemini
    experience_query_builder_model: Gemini


# See more models at https://ai.google.dev/gemini-api/docs/pricing
config = CvSaverConfig(
    job_offer_analyzer_model=Gemini(
        model="gemini-2.5-flash-preview-09-2025",
        retry_options=retry_config,
    ),
    experience_refiner_model=Gemini(
        model="gemini-2.5-flash-preview-09-2025",
        retry_options=retry_config,
    ),
    experience_critique_model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    experience_query_builder_model=Gemini(
        model="gemini-2.5-flash-lite-preview-09-2025",
        retry_options=retry_config,
    ),
)

## 2. 📝 Data Handling and Preprocessing

This section covers the loading, validation, and preparation of the candidate's CV data.

### 💾 CV Loading, Validation, and Display

These cells implement the loading of the JSON CV file and its validation using **Pydantic schemas** (`JsonResume`), ensuring data integrity and structure before it is consumed by the RAG system.

* **`get_json_resume()`**: Loads the raw JSON and uses `JsonResume.model_validate_json()` to strictly enforce the expected CV data structure (a core use of Pydantic).
* **Display/Formatting**: Iterates over the validated resume object, displaying the distinct experiences in a human-readable Markdown format using the custom `format()` method. This step is crucial for verifying the input data's state.

In [5]:
from pathlib import Path

from cv_enhancer.schemas import JsonResume


def get_json_resume() -> JsonResume:
    """Load a JSON resume from a file and validate it against the JsonResume schema.

    Returns:
        JsonResume: The validated JSON resume object.
    """
    with Path(JSON_RESUME_FILE).open(encoding="utf-8") as f:
        json_resume = JsonResume.model_validate_json(f.read())

    return json_resume

In [6]:
from rich import print as rprint

json_resume = get_json_resume()

rprint(json_resume)

JsonResume(
    basics=Basics(
        name='Andrea Perez Garcia',
        label='Software Engineer and Cloud Architect',
        image='',
        email='andrea.perez.dev@email.com',
        phone='(+34) 600 123 456',
        url=HttpUrl('https://www.linkedin.com/in/andreaperezdev'),
        summary='Professional with 10 years of experience in software development, specializing in scalable systems
architecture and migration to cloud environments. I seek to apply my experience in technical leadership and complex
project management, driving innovation and operational efficiency in a new challenge.',
        location=Location(
            address='Calle Ficticia 123',
            postalCode='28080',
            city='Madrid',
            countryCode='ES',
            region='Comunidad de Madrid'
        ),
        profiles=[
            Profile(
                network='LinkedIn',
                username='andreaperezdev',
                url=HttpUrl('https://www.linkedin.com/in/andreaperezdev')
            ),
            Profile(network='GitHub', username='aperez-dev', url=HttpUrl('https://github.com/aperez-dev'))
        ]
    ),
    work=[
        WorkItem(
            name='TechGlobal Solutions',
            position='Principal Solutions Architect',
            url=HttpUrl('https://www.techglobal.com/'),
            startDate=datetime.datetime(2023, 1, 1, 0, 0),
            endDate=None,
            summary='Technical leadership in the definition and implementation of microservices-based architectures
in AWS. Focused on cost optimization and resilience of critical systems.',
            highlights=[
                "Designed and implemented a 'lift-and-shift' cloud migration resulting in 20% operational 
savings.",
                'Mentored 5 development teams on DevOps and CI/CD best practices.'
            ],
            keywords=['AWS', 'Microservices', 'DevOps', 'Cloud Migration', 'Leadership']
        ),
        WorkItem(
            name='Innovación Digital S.A.',
            position='Senior Software Engineer',
            url=HttpUrl('https://www.innovaciondigital.com/'),
            startDate=datetime.datetime(2019, 6, 1, 0, 0),
            endDate=datetime.datetime(2022, 12, 31, 0, 0),
            summary='Full-stack development of high-traffic e-commerce platforms. Responsible for refactoring 
legacy code.',
            highlights=[
                'Reduced main API latency by 35% through SQL query optimization and caching.',
                'Implemented unit and integration tests, increasing code coverage to 90%.'
            ],
            keywords=['Full-stack', 'E-commerce', 'Performance Tuning', 'Refactoring', 'Testing']
        ),
        WorkItem(
            name='Global Data Systems',
            position='Backend Developer',
            url=HttpUrl('https://www.globaldata.net/'),
            startDate=datetime.datetime(2016, 8, 1, 0, 0),
            endDate=datetime.datetime(2019, 5, 31, 0, 0),
            summary='Involvement in the development of RESTful APIs for financial services.',
            highlights=[
                'Development of new features and maintenance of the client database.',
                'Collaboration on technical documentation for the APIs.'
            ],
            keywords=['Backend Development', 'REST API', 'SQL', 'Documentation']
        ),
        WorkItem(
            name='Startup Ágil',
            position='Junior Developer',
            url=HttpUrl('https://www.startupagil.co/'),
            startDate=datetime.datetime(2015, 1, 1, 0, 0),
            endDate=datetime.datetime(2016, 7, 31, 0, 0),
            summary='First professional role focused on frontend development and customer support.',
            highlights=[
                'Utilization of JavaScript frameworks for user interface development.',
                'Resolution of critical production bugs.'
            ],
            keywords=['Frontend', 'JavaScript', 'UI/UX', 'Bug Fixing']
        ),
        W

In [7]:
from IPython.display import Markdown
from IPython.display import display as ip_display

from cv_enhancer.schemas.json_resume._abc import JsonResumeFormattableBaseModel

experencies: list[JsonResumeFormattableBaseModel] = []
different_experencies = {}
for exp in json_resume.iter_over_formatables():
    experencies.append(exp)
    different_experencies[type(exp)] = exp

for exp in different_experencies.values():
    ip_display(Markdown(exp.format()))


## Company: Universidad Tecnológica

### Position: Research Intern

- Summary:
    > Collaboration on a machine learning project for optimizing search algorithms.

- Highlights:
    - Collection and preprocessing of large datasets.
    - Performance analysis of algorithms in Python.


- Keywords: Machine Learning, Data Analysis, Python, Research



## Organization: Club de Programación Juvenil

### Position: Instructor

- Summary:
    > Teaching the fundamentals of programming to young people aged 12 to 16.

- Highlights:
    - Organized a hackathon for students.


- Keywords: Education, Instruction, Youth Programs, Hackathon



## Certificate: Advanced English Certificate (C2)

###  Issuer: Cambridge Assessment English

- Summary:
    > C2 level in the Common European Framework of Reference for Languages.

- Keywords: English, C2 Level, Language Skills



## Skill: Databases

- Level: Intermediate

- Keywords: PostgreSQL, MySQL, MongoDB, Redis



## Project: Stock Data Visualizer

- Description:
    > Desktop tool (Electron) for visualizing real-time stock trends, consuming data from a public API.

- Highlights:
    - Used charting libraries (Chart.js) for data representation.
    - Implemented dark/light mode.


- Keywords: Electron, JavaScript, Chart.js, Finance


### 🔢 RAG Architecture: Custom Gemini Embedding Function

This is a critical architectural cell defining the **RAG (Retrieval-Augmented Generation)** component. It implements a custom `GeminiEmbeddingFunction` for ChromaDB, which is necessary to align the embedding generation with the Google AI SDK best practices.

* **Retry Decorator (`@retry.Retry`)**: Applies resilience to the embedding call, handling potential API errors (429/503) specifically.
* **Task Type (Design)**: Explicitly sets the `task_type` (`RETRIEVAL_DOCUMENT` vs. `RETRIEVAL_QUERY`) based on whether the input is for indexing (the CV) or searching (the query). This ensures **optimal embedding quality** for the retrieval task.

In [8]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google import genai
from google.api_core import retry
from google.genai import types
from google.genai.errors import APIError

client = genai.Client()


# Define a helper to retry when per-minute quota is reached.
def is_retriable(e):
    return isinstance(e, APIError) and e.code in {429, 503}


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    EMBEDDING_MODEL = "models/text-embedding-004"
    OUTPUT_DIM = 768

    def __init__(
        self,
        document_mode: bool = True,
        embedding_model: str | None = None,
        output_dim: int | None = None,
    ):
        self.document_mode = document_mode
        self.embedding_model = embedding_model
        self.output_dim = output_dim

    @property
    def task_type(self) -> str:
        return "RETRIEVAL_DOCUMENT" if self.document_mode else "RETRIEVAL_QUERY"

    @retry.Retry(predicate=is_retriable)
    @t.override
    def __call__(self, input: Documents) -> Embeddings:
        response = client.models.embed_content(
            model=self.embedding_model or self.EMBEDDING_MODEL,
            contents=input,  # type: ignore
            config=types.EmbedContentConfig(
                task_type=self.task_type,
                output_dimensionality=self.output_dim or self.OUTPUT_DIM,
            ),
        )
        return [e.values for e in response.embeddings]  # type: ignore

### 🗃️ ChromaDB Initialization and RAG Tool Definition

These cells initialize the Vector Database and define the core RAG function that acts as a **Tool** for the agent system.

* **Database Creation**: Initializes the `chromadb.Client()` and creates/gets the `cv_embeddings_v3` collection, injecting the custom `embed_fn`.
* **Indexing**: Adds all formatted experiences from the CV into the ChromaDB collection. Each experience's text is the `document`, and metadata (like `item_type`) is attached for filtering/categorization.
* **`retrieve_experiences_by_query()` (Agent Tool)**: Defines the function that interfaces with the ChromaDB. This function is essential as it takes the LLM-generated query and returns the most semantically relevant experiences, forming the **Knowledge Base** for the enhancement agents. It manages the `document_mode` switch internally.

In [9]:
import chromadb

embed_fn = GeminiEmbeddingFunction(document_mode=True)

DB_NAME = "cv_embeddings_" + JSON_RESUME_FILE.stem

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(
    documents=[e.format() for e in experencies],
    ids=[e.get_id() for e in experencies],
    metadatas=[{"item_type": e.item_type} for e in experencies],
)

In [10]:
import json

from chromadb.base_types import Metadata

ValidExperienceKeys = t.Literal[
    "work",
    "volunteer",
    "certificates",
    "projects",
    "skills",
    "interests",
]


class ExperienceRetrieval(t.TypedDict):
    exp_id: str
    exp_type: ValidExperienceKeys
    description: str
    distance: float


class ExperienceRetrievalResult(t.TypedDict, total=False):
    status: str
    retrieved_experiences: list[ExperienceRetrieval]
    message: str


def retrieve_experiences_by_query(
    query: str,
    n_results: int,
) -> ExperienceRetrievalResult:
    """Retrieve experiences from the ChromaDB based on a query string.
    The embedding function will be switched to query mode during this operation.

    Args:
        query (str): The query string to search for.
        n_results (int): The number of top results to retrieve. If the value
            is less than or equal to zero, all experiences will be retrieved.

    Returns:
        dict[str, str]: A dictionary containing the status and retrieved experiences.

    Example:
        >>> retrieve_experiences_by_query(
        ...     query="Python, SQL, Machine Learning",
        ...     n_results=2,
        ... )
        {
            "status": "success",
            "retrieved_experiences": [
                {
                    "exp_id": "work_1",
                    "description": "Developed data pipelines using Python and SQL...",
                    "exp_type": "work",
                    "distance": 0.12345
                },
                {
                    "exp_id": "project_3",
                    "description": "Implemented machine learning models using Python...",
                    "exp_type": "project",
                    "distance": 0.23456
                }
            ]
        }
    """
    # Switch to query mode when generating embeddings.
    embed_fn.document_mode = False

    if db.count() == 0:
        return {
            "status": "error",
            "message": "The database is empty. No experiences to retrieve.",
        }

    # Search the Chroma DB using the specified query.
    if n_results <= 0:
        n_results = db.count()

    try:
        result = db.query(query_texts=[query], n_results=n_results)

        documents: list[str] = []
        ids: list[str] = []
        metadatas: list[Metadata] = []
        distances: list[float] = []

        if result["documents"]:
            documents = result["documents"][0]

        if result["ids"]:
            ids = result["ids"][0]

        if result["metadatas"]:
            metadatas = result["metadatas"][0]

        if result["distances"]:
            distances = result["distances"][0]

        retrived_experiences = [
            ExperienceRetrieval(
                exp_type=str(metadata["item_type"]),  # type: ignore
                description=passage,
                exp_id=exp_id,
                distance=distance,
            )
            for exp_id, passage, metadata, distance in zip(
                ids, documents, metadatas, distances, strict=False
            )
        ]

        return {
            "status": "success",
            "retrieved_experiences": retrived_experiences,
        }
    except Exception as e:
        return {
            "status": "error",
            "message": str(e),
        }

## 3. 🤖 Agent Definition and Orchestration
This is the core of the solution, defining the roles, prompts, and the overall sequential and parallel multi-agent flow.

This is the architecture diagram:

![Architecture Diagram](../imgs/resume_enhancement_pipeline.svg)

### 1. 📝 Agent Chain Start: Job Offer Analyzer Agent

This agent is the **entry point** for processing the job offer. Its primary role is **Data Extraction and Structuring**, converting unstructured job text into actionable, schema-compliant data.

* **Pydantic Schema (`JobOfferSummarized`)**: Enforces a strict, structured output format for the job summary, ensuring downstream agents receive clean, typed input (requirements, tech stack).
* **Agent Role (`Agent`)**: Defined using the Google AI SDK, with a clear `instruction` to be concise, accurate, and maintain the original language.
* **`output_schema`**: The Pydantic schema is enforced here, making this a **Reliable Extractor Agent**.

In [11]:
from pydantic import BaseModel, Field


class JobOfferSummarized(BaseModel):
    job_description: str = Field(
        ...,
        description="Concise description of the job",
    )
    requirements: list[str] = Field(
        ...,
        description="Key requirements listed clearly",
    )
    tech_stack: list[str] = Field(
        ...,
        description="Technologies and tools mentioned",
    )

    __EXAMPLE__: t.ClassVar = {
        "job_description": "Develop and maintain web applications.",
        "requirements": ["Python", "Django", "REST APIs"],
        "tech_stack": ["AWS", "Docker", "PostgreSQL"],
    }

In [12]:
from google.adk.agents import Agent

PROMPT = f"""
You are a Job Offer Summarizer. Your only task is to read the provided job offer
and extract the key information such as the description, requirements, stack
and provide a concise summary. Do not add any additional information
(such as benefits, the title, the company...) or opinions. Try to summarize
in 300 words maximum.

Try to be as specific as possible when extracting the tech stack.

Here is an example of the output format:
---
{json.dumps(JobOfferSummarized.__EXAMPLE__, indent=2)}
---

You MUST RETURN the output in the EXACT FORMAT as shown above, without any additional text.

Mantain the original language of the job offer.
"""

job_offer_analyzer_agent = Agent(
    name="JobOfferAnalyzerAgent",
    model=config.job_offer_analyzer_model,
    description=(
        "An agent that summarizes job offers by extracting key information and"
        " presenting it clearly to assist job seekers in understanding"
        " the opportunities."
    ),
    instruction=PROMPT,
    output_key="summarized_job_offer",
    output_schema=JobOfferSummarized,
)

print("✅ Job Offer Summarizer Agent defined.")  # noqa: T201

✅ Job Offer Summarizer Agent defined.


### 2. 📝 Agent Chain Step: Experience Query Builder Agent

This agent's task is to translate the structured job requirements into an optimal, dense **search query string** for the ChromaDB vector store.

* **Input**: `{{summarized_job_offer}}` (from the previous agent).
* **Output**: The `search_query` string (e.g., "Python, SQL, Machine Learning, Data Mining...").
* **Role**: It bridges the language model's comprehension capabilities (understanding job requirements) with the RAG system's vector retrieval mechanism.

In [13]:
def limit_experiences_by_type(
    experiences: list[ExperienceRetrieval],
    limits: ExperienceLimits,
) -> list[ExperienceRetrieval]:
    """Limit the number of experiences by type.

    Args:
        experiences (list[ExperienceRetrieval]): The list of experiences to limit.
        limits (ExperienceLimits): The limits for each experience type.

    Returns:
        list[ExperienceRetrieval]: The limited list of experiences.
    """
    from collections import Counter  # noqa: PLC0415

    counts: Counter[str] = Counter()
    limited_experiences: list[ExperienceRetrieval] = []

    for exp in experiences:
        exp_type: ValidExperienceKeys = exp["exp_type"]  # type: ignore
        if exp_type in limits and counts[exp_type] < limits[exp_type]:
            limited_experiences.append(exp)
            counts.update([exp_type])

    return limited_experiences

In [14]:
PROMPT = f"""
You are a Query Builder Agent. Your task is to construct a search query
that will help retrieve relevant experiences from a resume database based
on the provided job requirements and technology stack.

Job offer summary:
{{summarized_job_offer}}

Please ensure that the query is tailored to the specific technologies
and skills mentioned in the job description. Additionally, consider the
candidate's experience level and relevant certifications.

The output should be a concise search query string that effectively captures
the key requirements and technologies needed for the job.

Consider the following when building the query:
- The database is a vector database containing experiences from resumes,
    including work experience, volunteer work, certifications, projects,
    skills, and interests.
- The documents were stored using the mode `RETRIEVAL_DOCUMENT` of the
    model `{GeminiEmbeddingFunction.EMBEDDING_MODEL}`.
- The query will be of type `RETRIEVAL_QUERY` and should be compatible
    with the same model.
- It is not necessary to include the year of experience or seniority level
    in the query.
- Return ONLY the search query string as the output, without any additional
    commentary or formatting.
"""

experience_query_builder_agent = Agent(
    name="ExperienceQueryBuilderAgent",
    model=config.experience_query_builder_model,
    description="""An agent that builds search queries to retrieve relevant experiences
    from a resume database based on job requirements and technology stack.""",
    instruction=PROMPT,
    output_key="search_query",
)

### 3. 📝 Agent Chain Step: Custom Experience Retrieval Agent

This custom agent (`BaseAgent` subclass) executes the RAG retrieval operation outside of a standard LLM call, ensuring that the process is efficient and integrated directly into the agent workflow.

* **Custom Agent**: Subclassing `BaseAgent` is used because this agent executes a **Python Tool** (`retrieve_experiences_by_query`) rather than an LLM call.
* **Input**: Retrieves the `search_query` from the session state (placed by the `ExperienceQueryBuilderAgent`).
* **Output**: Saves the `retrieved_experiences` (filtered and limited by `EXPERIENCE_LIMITS`) into the session state for the next agents.

In [15]:
from google.adk.agents import BaseAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.events import Event


class ExperienceRetrieverAgent(BaseAgent):
    """An agent that retrieves the most relevant experiences and saves them in the session state."""

    experience_limits: ExperienceLimits | None = None

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        logging.info(f"[{self.name}] Starting Experience Retrieval Agent")

        # Get the search query from the session state
        query = ctx.session.state.get("search_query", None)
        if query is None:
            logging.error(f"[{self.name}] No search query found in session state.")
            return

        # Retrieve the experiences using the retrieve_experiences_by_query tool
        #  from the session state
        retrieved_exps_result = retrieve_experiences_by_query(query=query, n_results=0)

        if retrieved_exps_result.get("status") != "success":
            logging.error(
                f"[{self.name}] Failed to retrieve experiences:"
                f" {retrieved_exps_result.get('message')}"
            )
            return
        retrieved_exps = retrieved_exps_result["retrieved_experiences"]

        # Limit the number of experiences if limits are provided
        if self.experience_limits:
            retrieved_exps = limit_experiences_by_type(
                retrieved_exps,
                limits=self.experience_limits,
            )

        # Sort experiences by exp_id in descending order to display
        #  the most recent ones first.
        retrieved_exps.sort(key=lambda x: x["exp_id"], reverse=True)

        # Save the retrieved experiences in the session state
        ctx.session.state["retrieved_experiences"] = retrieved_exps

        logging.info(
            f"[{self.name}] Retrieved {len(retrieved_exps)} experiences for"
            " enhancement."
        )
        logging.debug(f"[{self.name}] Experiences: {retrieved_exps}")

        yield Event(author=self.name)


experience_retriever_agent = ExperienceRetrieverAgent(
    name="ExperienceRetrieverAgent",
    experience_limits=EXPERIENCE_LIMITS,
)

### 🔄 Refinement Loop Architecture (Iterative Self-Correction)

This core architectural pattern implements a **Refinement Loop** using the Google AI SDK's `LoopAgent`. This mechanism ensures **iterative self-correction** by engaging a dedicated **Critic Agent** to evaluate the initial experience enhancement draft.

#### 📦 Enhancement Output Schemas (Pydantic Design)

This block defines the structured output schemas for the **Experience Enhancement Agents**. Using inheritance and specific fields for `work`, `projects`, and `skills`, it ensures the enhanced CV output is consistently formatted and validated.

* **Structured Output**: Defines nested schemas (`SummaryKeywordsOutputSchema`, etc.) to enforce the required fields: `summary`, `highlights`, and `keywords`. This is critical for generating a CV that meets professional formatting standards.

In [16]:
class KeywordsOutputSchema(BaseModel):
    keywords: list[str] = Field(
        ...,
        description="List of relevant keywords or skills.",
    )


class SummaryKeywordsOutputSchema(KeywordsOutputSchema):
    summary: str = Field(
        ...,
        description="A brief summary of the experience.",
    )


class SummaryHighlightsKeywordsOutputSchema(SummaryKeywordsOutputSchema):
    highlights: list[str] = Field(
        ...,
        description="Key achievements or highlights of the experience.",
    )


OUTPUT_SCHEMAS = {
    "work": SummaryHighlightsKeywordsOutputSchema,
    "volunteer": SummaryHighlightsKeywordsOutputSchema,
    "certificates": SummaryKeywordsOutputSchema,
    "projects": SummaryHighlightsKeywordsOutputSchema,
    "skills": KeywordsOutputSchema,
    "interests": KeywordsOutputSchema,
}


EXAMPLES = {
    "work": {
        "summary": "Description…",
        "highlights": ["Started the company"],
        "keywords": ["leadership", "entrepreneurship"],
    },
    "volunteer": {
        "summary": "Description…",
        "highlights": ["Volunteered at local shelter"],
        "keywords": ["community", "helping others"],
    },
    "certificates": {
        "summary": "Description…",
        "keywords": ["certification", "achievement"],
    },
    "projects": {
        "summary": "Description…",
        "highlights": ["Developed a web app"],
        "keywords": ["Python", "Django", "AWS"],
    },
    "skills": {
        "keywords": ["Python", "Machine Learning", "Data Analysis"],
    },
    "interests": {
        "keywords": ["hiking", "photography", "travel"],
    },
}

#### 💬 Agent Prompt Templates (Behavior Definition)

This cell defines the core instruction sets for the **Enhancement** and **Critic** agents, implementing the **Refinement Loop** strategy.

* **`PROMPT_INITIAL_EXP_ENHANCEMENT`**: Sets the instructions for the first-pass enhancement, emphasizing alignment with the `{{summarized_job_offer}}` and adherence to the Pydantic output format.
* **`PROMPT_CRITIC` (Refinement Core)**: Defines the role of the critic. This agent performs a **Self-Correction/Iterative Improvement** function. Its output is concise ("APPROVED" or specific suggestions), designed to be used as input for the next agent iteration.
* **`PROMPT_REFINER_EXP_ENHANCEMENT`**: Instructs the refining agent to incorporate the `{{critique_{agent_id}}}` feedback to improve the previous draft. The use of the `Original Experience Description` helps prevent "hallucination" by grounding the agent in the source text.

In [17]:
PROMPT_INITIAL_EXP_ENHANCEMENT = """
You are an experience enhancer. Your task is to improve the provided
description of experience to better align it with the job posting summary.

The main goal is to ensure that the experience description highlights
the skills, technologies, and accomplishments that are most relevant to
the job requirements and technology stack mentioned in the job posting.

Job Posting Summary:
{{summarized_job_offer}}

Experience Description:
{{experience_{agent_id}}}

EXAMPLE:
{example}

YOU MUST RETURN the output in the EXACT FORMAT as shown in the example,
without any additional text. NOT FOLLOWING THE FORMAT WILL CAUSE ERRORS.

OUTPUT RULES:
- The summary/description must be concise and focused on relevant skills
    and accomplishments. Avoid unnecessary details. Try to keep it
    in 1-2 sentences.
- The highlights must be specific achievements or contributions,
    quantifiable where possible. Try to include 2-3 highlights. Each highlight
    should start with a strong action verb. Try to keep it in a sentence each.
- The keywords must include relevant technologies, tools, and skills.
    Try to include 3-5 keywords.
- Do not invent details. Only use the information provided in the
    experience description.
- Mantain the original language of the experience description.
"""

PROMPT_CRITIC = """
You are a constructive critic. Your task is to review the provided
draft of experience and provide feedback on how well it aligns with the
job posting summary. Identify areas of improvement, suggest enhancements,
and highlight any discrepancies between the draft and the job requirements.

Job Posting Summary:
{{summarized_job_offer}}

Experience Draft:
{{enhanced_experience_{agent_id}}}

- If the draft is well-aligned, you MUST respond with the exact phrase: 'APPROVED'
- Otherwise, provide 1-3 specific, actionable suggestions for improvement,
    of the draft to better align it with the job posting summary.
- Be concise and specific in your feedback. Think about your suggestions
    will be used for other agent to improve the experience draft.
"""

PROMPT_REFINER_EXP_ENHANCEMENT = """
You are an experience enhancer. Your task is to improve the provided
description of experience to better align it with the job posting summary.
The main goal is to ensure that the experience description highlights
the skills, technologies, and accomplishments that are most relevant to
the job requirements and technology stack mentioned in the job posting.

Job Posting Summary:
{{summarized_job_offer}}

Original Experience Description:
{{experience_{agent_id}}}

Current Experience Draft:
{{enhanced_experience_{agent_id}}}

Critique Feedback:
{{critique_{agent_id}}}

EXAMPLE:
{example}

YOU MUST RETURN the output in the EXACT FORMAT as shown in the example,
without any additional text. NOT FOLLOWING THE FORMAT WILL CAUSE ERRORS.

OUTPUT RULES:
- Use the critique feedback to make specific improvements
    to the experience draft to better align it with the job posting summary.
- The summary/description must be concise and focused on relevant skills
    and accomplishments. Avoid unnecessary details. Try to keep it
    in 1-2 sentences.
- The highlights must be specific achievements or contributions,
    quantifiable where possible. Try to include 2-3 highlights. Each highlight
    should start with a strong action verb. Try to keep it in a sentence each.
- The keywords must include relevant technologies, tools, and skills.
    Try to include 3-5 keywords.
- Do not invent details. Only use the information provided in the
    experience description.
- Mantain the original language of the experience description.
"""

#### 🔧 Refinement Loop Control Tool

This simple function is defined as a **Tool** that the Refiner Agent can call. It serves as the explicit mechanism to **break the iterative refinement loop** when the Critic Agent responds with "APPROVED." This is a key design pattern for controlled, tool-augmented loops.

In [18]:
from google.adk.events import EventActions


class CheckStatusAndEscalateAgent(BaseAgent):
    """An agent that checks the status of the experience refinement
    and escalates if approved."""

    agent_id: str

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        logging.info(f"[{self.name}] Starting Check Status And Escalate Agent")

        # Get the critique result from the session state
        critique_result = ctx.session.state.get(f"critique_{self.agent_id}", "fail")
        should_stop = critique_result.lower() == "approved"

        logging.debug(
            f"[{self.name}] Critique result: {critique_result},"
            f" should_stop={should_stop}"
        )
        yield Event(author=self.name, actions=EventActions(escalate=should_stop))

#### 🏭 Agent Factory: Multi-Agent Refinement System Creator

This crucial function (`experience_refinament_agent_factory`) acts as a **factory** to dynamically create a complete **Sequential Agent** (which includes a **Loop Agent**) for *each individual experience* retrieved from the RAG step.

* **Sequential Flow**: `InitialEnhancement -> LoopAgent`.
* **Loop Agent**: Manages the iterative refinement flow: `Critic Agent -> Refiner Agent`.
* **Meaningful Use of Agents**: This is the core architectural feature:
    * **Specialization**: The system uses two specialized LLMs (Flash for generation, Pro for critique) within a single loop.
    * **Self-Correction**: The Loop Agent implements a *self-correction feedback mechanism*, significantly improving output quality over a single-pass LLM call.
    * **Tool Use**: The Refiner Agent is equipped with the `exit_loop` tool, allowing it to programmatically terminate the iteration when the goal is met.

In [19]:
import uuid

from google.adk.agents import LoopAgent, SequentialAgent


def experience_refinament_agent_factory(
    exp_type: ValidExperienceKeys, agent_id: str | None = None, max_iterations: int = 1
) -> SequentialAgent:
    # Use a unique agent ID if not provided
    if agent_id is None:
        agent_id = str(uuid.uuid4())[:8]

    example = json.dumps(EXAMPLES[exp_type], indent=2)
    schema_output = OUTPUT_SCHEMAS[exp_type]

    initial_experience_draft_agent = Agent(
        name=f"InitialExperienceDraftAgent_{agent_id}",
        model=config.experience_refiner_model,
        description=(
            "An agent that enhances a first draft of experience"
            " descriptions to better align them with job requirements and"
            " technology stack."
        ),
        instruction=PROMPT_INITIAL_EXP_ENHANCEMENT.format(
            agent_id=agent_id, example=example
        ),
        output_key=f"enhanced_experience_{agent_id}",
        output_schema=schema_output,
    )

    experience_critique_agent = Agent(
        name=f"ExperienceCritiqueAgent_{agent_id}",
        model=config.experience_critique_model,
        description=(
            "An agent that critiques and provides feedback on a draft of experience."
        ),
        instruction=PROMPT_CRITIC.format(agent_id=agent_id),
        output_key=f"critique_{agent_id}",
    )

    experience_refiner_agent = Agent(
        name=f"ExperienceRefinerAgent_{agent_id}",
        model=config.experience_refiner_model,
        description=(
            "An agent that enhances a draft of experience descriptions based on"
            " feedback from a critic agent to better align them with job"
            " requirements and technology stack."
        ),
        instruction=PROMPT_REFINER_EXP_ENHANCEMENT.format(
            agent_id=agent_id, example=example
        ),
        output_key=f"enhanced_experience_{agent_id}",
        output_schema=schema_output,
    )

    refinement_loop_agent = LoopAgent(
        name=f"RefinementLoopAgent_{agent_id}",
        sub_agents=[
            experience_critique_agent,
            CheckStatusAndEscalateAgent(
                name=f"CheckStatusAndEscalateAgent_{agent_id}",
                agent_id=agent_id,
            ),
            experience_refiner_agent,
        ],
        max_iterations=max_iterations,
    )

    experience_enhancement_sequence = SequentialAgent(
        name=f"ExperienceRefinementSequence_{agent_id}",
        sub_agents=[
            initial_experience_draft_agent,
            refinement_loop_agent,
        ],
    )

    return experience_enhancement_sequence

### 4. 📝 Agent Chain Step: Experience Enhancement Orchestrator Agent (Parallel Execution)

This custom agent is responsible for **orchestrating the parallel execution** of all individual experience enhancement agents.

* **Parallelism (Architecture)**: It dynamically creates a `ParallelAgent` containing the specialized `ExperienceEnhancementAgent` for *each* retrieved experience. This significantly reduces total latency by running multiple LLM calls concurrently.
* **Data Preparation**: It saves individual experience descriptions into the session state using unique keys (`experience_{agent_id}`) so that each parallel enhancement agent has access only to its required input.

In [20]:
class ExperienceEnhancementOrchestratorAgent(BaseAgent):
    """An agent that aggregates enhanced experiences from sub-agents."""

    max_refinement_iterations: int = 1

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        logging.info(
            f"[{self.name}] Starting Experiences Enhancement Aggregator Agent."
        )

        retrived_exps: list[ExperienceRetrieval] = ctx.session.state.get(
            "retrieved_experiences", []
        )
        if not retrived_exps:
            logging.error(
                f"[{self.name}] No retrieved experiences found in session state."
            )
            return

        # Create a parallel agent to enhance each experience concurrently
        tagged_experiences: dict[str, ExperienceRetrieval] = {}
        parallel_sub_agents: list[SequentialAgent] = []
        for exp in retrived_exps:
            # We will tag each agent with the experience ID.
            agent_id = exp["exp_id"].replace(".", "_")

            # Save the experience description in the session state,
            #  to be used by the experience enhancement agents.
            ctx.session.state[f"experience_{agent_id}"] = exp["description"]

            # Create an experience enhancement agent for each experience
            enh_agent = experience_refinament_agent_factory(
                exp_type=exp["exp_type"],
                agent_id=agent_id,
                max_iterations=self.max_refinement_iterations,
            )

            # Finally, tag the experience for reference
            tagged_experiences[agent_id] = exp
            parallel_sub_agents.append(enh_agent)

        parallel_enhancement_agent = ParallelAgent(
            name="ParallelExperienceEnhancementAgent",
            sub_agents=parallel_sub_agents,
        )

        # Save tagged experiences in the session state for later reference
        ctx.session.state["tagged_experiences"] = tagged_experiences

        # Run the parallel enhancement agent
        async for event in parallel_enhancement_agent.run_async(ctx):
            yield event


experience_enhancement_orchestrator_agent = ExperienceEnhancementOrchestratorAgent(
    name="ExperienceEnhancementOrchestratorAgent",
    max_refinement_iterations=MAX_REFINEMENT_ITERATIONS,
)

### 5. 📝 Agent Chain Step: Enhanced CV Assembly Agent (Final Output)

The final agent in the sequence, responsible for aggregating the results of all preceding agents and compiling the final, enhanced CV document.

* **Aggregation**: Gathers the final, structured enhanced content (`enhanced_experience_{agent_id}`) from the session state.
* **Compilation**: Merges the unchanged basic CV data with the newly enhanced experience sections into a single, validated `JsonResume` object.
* **Output**: Saves the final `enhanced_cv` object to the session state and outputs the JSON string.

In [21]:
import logging

from google.adk.agents import ParallelAgent


class EnhancedCvAssemblyAgent(BaseAgent):
    """This agent enhances a CV based on the job offer summary and
    retrieved experiences.
    """

    json_resume_template: JsonResume

    @t.override
    async def _run_async_impl(self, ctx: InvocationContext) -> t.AsyncGenerator[Event]:
        logging.info(f"[{self.name}] Starting CV Builder Agent.")

        # Get tagged experiences from the session state
        tagged_experiences = t.cast(
            dict[str, ExperienceRetrieval],
            ctx.session.state.get("tagged_experiences", {}),
        )

        # Get enhanced experiences from the session state
        enhanced_experiences: dict[str, dict[str, t.Any]] = {}
        for agent_id in tagged_experiences:
            enhanced_exp = t.cast(
                dict[str, t.Any],
                ctx.session.state.get(f"enhanced_experience_{agent_id}"),
            )
            enhanced_experiences[agent_id] = enhanced_exp

        # Build a json resume with the enhanced experiences
        json_resume_enhanced: dict[str, dict | list] = {}

        # Include basics from the template. It's assumed to be unchanged.
        if basics := self.json_resume_template.basics:
            json_resume_enhanced["basics"] = basics.model_dump()

        # Modify experiences with the enhanced versions
        for agent_id, enhanced_exp in enhanced_experiences.items():
            exp_id = tagged_experiences[agent_id]["exp_id"]
            exp_type = tagged_experiences[agent_id]["exp_type"]
            if exp_type not in json_resume_enhanced:
                json_resume_enhanced[exp_type] = []

            experience_obj = self.json_resume_template.find_experience(exp_id)
            if experience_obj:
                experience_dict = experience_obj.model_dump()
                for field, value in enhanced_exp.items():
                    experience_dict[field] = value
                json_resume_enhanced[exp_type].append(experience_dict)  # type: ignore

        enhanced_cv = JsonResume(**json_resume_enhanced)
        ctx.session.state["enhanced_cv"] = enhanced_cv

        yield Event(
            author=self.name,
            content={"parts": [{"text": enhanced_cv.model_dump_json(indent=2)}]},
        )


enhanced_cv_assembly_agent = EnhancedCvAssemblyAgent(
    name="EnhancedCvAssemblyAgent",
    json_resume_template=get_json_resume(),
)

### 🚀 Root Agent Orchestration (Main Pipeline)

This cell defines the **Root Sequential Agent**, establishing the definitive, end-to-end flow of the entire CV enhancement process.

* **Sequential Flow**: Clearly defines the order of operations, ensuring that the output of one agent (e.g., `job_offer_analyzer_agent`) becomes the necessary input for the next (e.g., `experience_query_builder_agent`).
* **Architecture Review**: This definition confirms the overall solution architecture: **Sequential Agents for Pipeline Flow** + **Parallel Agents for Concurrent Processing** + **Loop Agents for Self-Correction/Refinement**.

In [22]:
from google.adk.agents import SequentialAgent

root_agent = SequentialAgent(
    name="CvEnhancerFlowAgent",
    sub_agents=[
        job_offer_analyzer_agent,
        experience_query_builder_agent,
        experience_retriever_agent,
        experience_enhancement_orchestrator_agent,
        enhanced_cv_assembly_agent,
    ],
)

## 4. 📈 Execution and Results

### 🏃 Runner Initialization and Logging

This block initializes the `Runner`, which is the execution environment for the agent system, and sets up session management and logging.

* **`InMemorySessionService`**: Used for managing the shared `session.state` across all agents in the pipeline (how data like `search_query` and `enhanced_experience` is passed).
* **`LoggingPlugin`**: Ensures detailed tracking of the agent's internal reasoning, which is essential for debugging and performance analysis in a multi-agent system.

In [23]:
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

session_service = InMemorySessionService()

runner = Runner(
    agent=root_agent,
    app_name="cv_enhancer_app",
    session_service=session_service,
    plugins=[
        LoggingPlugin(),
    ],
)

print("✅ Runner created.")  # noqa: T201

✅ Runner created.


#### ✅ Execution and Final Result

* **`runner.run_debug(JOB_OFFER_TEXT)`**: Executes the `CvEnhancerRootAgent` on the input job offer text. The `run_debug` method provides verbose output, crucial for understanding the flow and state changes in a complex multi-agent execution.
* **Final Output**: Extracts and validates the enhanced CV from the final response, confirming that the entire pipeline successfully executed and produced a structured, enhanced `JsonResume` object ready for review.

In [24]:
response = await runner.run_debug(JOB_OFFER_TEXT)


 ### Created new session: debug_session_id

User > 
We are still looking for talent… and we would love for you to join our team!

For over 25 years, UST has worked alongside the world's best companies to make a real impact through business transformation. Driven by technology, inspired by people, and guided by our purpose, UST supports clients from design to implementation. Together, with more than 30,000 employees in 30 countries, we build to create limitless impact, reaching billions of lives in the process.


We are looking for an AI Engineer, to join a strategic project supporting data platform modernization.



UST is looking for a candidate with strong Python expertise, proven experience building applications using LLMs, and hands-on exposure to agentic AI frameworks.



What We're Looking For:

Experience: 6 to 8 years of professional experience.
Language: Advanced English B2 - C1
Strong programming expertise in Python.
Proven experience building applications using any LLMs.
1 

In [25]:
last_response = response[-1]

enhanced_cv = None
if (content := last_response.content) and (parts := content.parts):
    part = parts[0]
    if cv_json := part.text:
        enhanced_cv = JsonResume.model_validate_json(cv_json)

In [26]:
rprint(enhanced_cv)

JsonResume(
    basics=Basics(
        name='Andrea Perez Garcia',
        label='Software Engineer and Cloud Architect',
        image='',
        email='andrea.perez.dev@email.com',
        phone='(+34) 600 123 456',
        url=HttpUrl('https://www.linkedin.com/in/andreaperezdev'),
        summary='Professional with 10 years of experience in software development, specializing in scalable systems
architecture and migration to cloud environments. I seek to apply my experience in technical leadership and complex
project management, driving innovation and operational efficiency in a new challenge.',
        location=Location(
            address='Calle Ficticia 123',
            postalCode='28080',
            city='Madrid',
            countryCode='ES',
            region='Comunidad de Madrid'
        ),
        profiles=[
            Profile(
                network='LinkedIn',
                username='andreaperezdev',
                url=HttpUrl('https://www.linkedin.com/in/andreaperezdev')
            ),
            Profile(network='GitHub', username='aperez-dev', url=HttpUrl('https://github.com/aperez-dev'))
        ]
    ),
    work=[
        WorkItem(
            name='TechGlobal Solutions',
            position='Principal Solutions Architect',
            url=HttpUrl('https://www.techglobal.com/'),
            startDate=datetime.datetime(2023, 1, 1, 0, 0),
            endDate=None,
            summary='Provided technical leadership in defining and implementing resilient, cloud-native 
architectures in AWS, prioritizing platform modernization strategies essential for deploying advanced AI 
applications built with Python and LLMs.',
            highlights=[
                'Led technical definition and implementation of scalable, cloud-native architectures in AWS, 
enabling resilient deployment and management of Python-based LLM applications.',
                'Architected critical system solutions utilizing principles derived from agentic AI frameworks to 
enhance automation and resilience in cloud modernization efforts.',
                'Mentored multiple development teams on DevOps and CI/CD best practices, accelerating high-velocity
software deployments crucial for iterative AI/ML development cycles.'
            ],
            keywords=[
                'Python',
                'LLMs',
                'Agentic AI frameworks',
                'AWS',
                'Cloud platforms',
                'Architecture',
                'Leadership'
            ]
        ),
        WorkItem(
            name='Global Data Systems',
            position='Backend Developer',
            url=HttpUrl('https://www.globaldata.net/'),
            startDate=datetime.datetime(2016, 8, 1, 0, 0),
            endDate=datetime.datetime(2019, 5, 31, 0, 0),
            summary='Leveraged strong Python programming expertise to develop and implement robust RESTful API 
features for financial services, focusing on integrating sophisticated functionalities derived from AI application 
requirements.',
            highlights=[
                'Developed and implemented Python-based features for core APIs, providing exposure to building 
applications using Large Language Models (LLMs) to enhance service capability.',
                'Applied foundational concepts of agentic AI frameworks in feature design, accelerating the 
implementation of complex business logic within financial service applications.',
                'Maintained service deployment and scalability, gaining familiarity with cloud platforms for 
operational performance, and collaborated on technical documentation for implemented APIs.'
            ],
            keywords=['Python', 'LLMs', 'Agentic AI frameworks', 'Cloud platforms', 'API Development']
        ),
        WorkItem(
            name='Universidad Tecnológica',
            position='Research Intern',
            url=HttpUrl('https://www.utec.edu/'),
            startDate=datetime.datetime(2014, 6, 1, 0, 0),
       